In [1]:
# Dependencies
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import pymongo
from selenium import webdriver
from splinter import Browser

# NASA Mars News Scrape

In [2]:
# Import HTML Code
html = urllib.request.urlopen('https://mars.nasa.gov/news/').read()
soup = bs(html, 'html.parser')

mars = {}

In [3]:
# Print all title texts
news_title = soup.find_all('div', class_='content_title')
for title in news_title:
    news_tit = title.text.strip()
    mars.update({'newsTitle': news_tit})

In [4]:
# Print all paragraph texts
news_p = soup.find_all('div', class_='rollover_description_inner')
for p in news_p:
    text = p.text.strip()
    mars.update({'newsText': text})

# JPL Mars Space Images - Featured Image

In [5]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [6]:
# Visit the following URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')
articles = soup.find('a', class_ = 'button fancybox')
browser.click_link_by_partial_text('FULL')
xpath = '//*[@id="fancybox-lock"]/div/div[1]/img'

In [8]:
# Use splinter to Click the "Mars in natural color in 2007" image 
# to bring up the full resolution image
results = browser.find_by_xpath(xpath)
img = results['src']
mars.update({'featuredImage': img})

# Mars Weather

In [9]:
html = urllib.request.urlopen('https://twitter.com/marswxreport?lang=en').read()
soup = bs(html, 'html.parser')
weather = soup.find("div", class_="js-tweet-text-container").text
text = p.text.strip()
mars.update({'marsWeather': weather})

In [10]:
mars

{'featuredImage': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA09178_ip.jpg',
 'marsWeather': '\nSol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28\n',
 'newsText': 'On a part of "Vera Rubin Ridge" where rover-team researchers sought to determine whether dust coatings are hiding rocks\' hematite content, the Mast Camera (Mastcam) on NASA\'s Curiosity Mars rover took this image of a rock surface that had been brushed with the rover\'s Dust Removal Tool.',
 'newsTitle': "Martian Ridge Brings Out Rover's Color Talents"}

# Mars Facts

In [11]:
# Import HTML Code
url = 'https://space-facts.com/mars/'

In [12]:
tables = pd.read_html(url)
tables[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
df = tables[0]

In [14]:
df.columns = ['variable','measurements']

In [15]:
fact_list = df["variable"]
value_list = df["measurements"]
fact_value = zip(fact_list, value_list)
table_dic = {}
for fact, value in fact_value:
   table_dic[fact] = value
table_dic

{'Equatorial Diameter:': '6,792 km',
 'First Record:': '2nd millennium BC',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Polar Diameter:': '6,752 km',
 'Recorded By:': 'Egyptian astronomers',
 'Surface Temperature:': '-153 to 20 °C'}

In [16]:
for fact, value in fact_value:
   mars[fact] = value

# Mars Hemisperes

In [17]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "./chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [18]:
html = browser.html
soup = bs(html, 'html.parser')
hemispheres = soup.find('div', class_ = 'collapsible results')

In [19]:
title = []
img_url = []
hemispheres = ["Cerberus","Schiaparelli","Syrtis","Valles"]


In [20]:
for hemisphere in hemispheres:
    browser.click_link_by_partial_text(hemisphere) 
    html = browser.html
    soup = bs(html, 'html.parser')
    img = soup.find("div", class_="downloads").ul.li.a["href"]
    text = soup.body.find('h2').text
    mars.update({text:img})
    browser.click_link_by_partial_text('Back')

In [21]:
mars

{'Cerberus Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'Schiaparelli Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'Syrtis Major Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'Valles Marineris Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
 'featuredImage': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA09178_ip.jpg',
 'marsWeather': '\nSol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28\n',
 'newsText': 'On a part of "Vera Rubin Ridge" where rover-team researchers sought to determine whether dust coatings are hiding rocks\' hematite content, the Mast Camera (Mastcam) on NASA\'s Curiosity Mars rover took this image of a ro